In [2]:
# Speed comparison SH
import pandas as pd
import numpy as np
import glob
import os

path = np.array(glob.glob('/mnt/data/20201228/zt***/Logs_full/mdLog_SH_***'))
assert(len(path) == 3)
print(path[0])
print(path[2])

logSH = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
logSH1 = pd.read_csv(path[2], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
print("original data source type: ")
print(logSH["source"].unique())
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
fasterPerc = []
fasterTm25 = []
fasterTm = []
fasterTm75 = []
slowerTm25 = []
slowerTm = []
slowerTm75 = []
slowerPerc = []

display(logSH['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(logSH1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())



/mnt/data/20201228/zt_7401_20201228_10.235.16.168.tar.gz/Logs_full/mdLog_SH_20201228_0923.csv
/mnt/data/20201228/zt_7401_20201228_10.235.16.80.tar.gz/Logs_full/mdLog_SH_20201228_0923.csv
original data source type: 
[29]
new data source type: 
[29]
['09:24:06.000' '09:24:07.000' '09:24:08.000' ... '15:39:41.000'
 '15:39:44.000' '15:39:48.000']
['09:23:56.000' '09:23:57.000' '09:23:58.000' ... '15:39:26.000'
 '15:39:26.280' '15:39:29.000']


array([1])

array([1])

In [3]:
### set here ***
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]

num = 1
### set here ***
    
if num == 2:
    for i in range(0, num):
        in_dex = [16, 300, 852, 905]
        data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == np.sort(logSH['source'].unique())[i])]
        data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == np.sort(logSH1['source'].unique())[i])]
        data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
        data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
        n1 = len(data1_1["StockID"].unique())
        n2 = len(data2_1["StockID"].unique())

        print(n1)
        print(n2)
        if n1 != n2:
            sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
            data1_1 = data1_1[data1_1["StockID"].isin(sl)]
            data2_1 = data2_1[data2_1["StockID"].isin(sl)]
        for cols in ['cum_amount']:
            data1_1[cols] = data1_1[cols].round(2)
            data2_1[cols] = data2_1[cols].round(2)
        data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
        n1 = test["sequenceNo_x"].count()
        n2 = test["sequenceNo_y"].count()
        len1 = len(test)
        print(n1)
        print(n2)
        print(len1)
        print("-----------------------------------------------")
        if n2 < len1:
            display("test is not complete:")
            display(test[np.isnan(test["sequenceNo_y"])])
            print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        if (len1 == n2) & (n1 < len1):
            display("baseline is not complete:")
            display(test[np.isnan(test["sequenceNo_x"])])
            display(n2-n1)
            print((n2-n1)/n1)

        # speed check
        test1 = test[~np.isnan(test["sequenceNo_x"])]
        test1 = test1[~np.isnan(test1["sequenceNo_y"])]
        diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
        fasterPerc.append(sum(diff < 0)/len(diff))
        slowerPerc.append(sum(diff > 0)/len(diff))
        fasterTm25.append(np.quantile(abs(diff[diff < 0]), 0.25))
        fasterTm.append(np.quantile(abs(diff[diff < 0]), 0.5))
        fasterTm75.append(np.quantile(abs(diff[diff < 0]), 0.75))
        slowerTm25.append(np.quantile(abs(diff[diff > 0]), 0.25))
        slowerTm.append(np.quantile(abs(diff[diff > 0]), 0.5))
        slowerTm75.append(np.quantile(abs(diff[diff > 0]), 0.75))

        print(np.quantile(abs(diff[diff < 0]), 0.25))
        print(np.median(abs(diff[diff < 0])))
        print(np.quantile(abs(diff[diff < 0]), 0.75))
        print(np.quantile(diff[diff > 0], 0.25))
        print(np.median(diff[diff > 0]))
        print(np.quantile(diff[diff > 0], 0.75))
        print(sum(diff < 0)/len(diff))
        print(sum(diff > 0)/len(diff))
else:
    in_dex = [16, 300, 852, 905]
    data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == logSH['source'].max())]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == logSH1['source'].max())]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())

    print(n1)
    print(n2)
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for cols in ['cum_amount']:
        data1_1[cols] = data1_1[cols].round(2)
        data2_1[cols] = data2_1[cols].round(2)
    data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)

    # speed check
    test1 = test[~np.isnan(test["sequenceNo_x"])]
    test1 = test1[~np.isnan(test1["sequenceNo_y"])]
    diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
    fasterPerc.append(sum(diff < 0)/len(diff))
    slowerPerc.append(sum(diff > 0)/len(diff))
    fasterTm25.append(np.quantile(abs(diff[diff < 0]), 0.25))
    fasterTm.append(np.quantile(abs(diff[diff < 0]), 0.5))
    fasterTm75.append(np.quantile(abs(diff[diff < 0]), 0.75))
    slowerTm25.append(np.quantile(abs(diff[diff > 0]), 0.25))
    slowerTm.append(np.quantile(abs(diff[diff > 0]), 0.5))
    slowerTm75.append(np.quantile(abs(diff[diff > 0]), 0.75))

    print(np.quantile(abs(diff[diff < 0]), 0.25))
    print(np.median(abs(diff[diff < 0])))
    print(np.quantile(abs(diff[diff < 0]), 0.75))
    print(np.quantile(diff[diff > 0], 0.25))
    print(np.median(diff[diff > 0]))
    print(np.quantile(diff[diff > 0], 0.75))
    print(sum(diff < 0)/len(diff))
    print(sum(diff > 0)/len(diff))    
    

in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]

display(data1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(data2['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())


### set here ***
columns = ["StockID", "cum_volume", "close", "openPrice"]
### set here ***

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

for cols in ["cum_amount"]:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])


# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
fasterPerc.append(sum(diff < 0)/len(diff))
slowerPerc.append(sum(diff > 0)/len(diff))
fasterTm25.append(np.quantile(abs(diff[diff < 0]), 0.25))
fasterTm.append(np.quantile(abs(diff[diff < 0]), 0.5))
fasterTm75.append(np.quantile(abs(diff[diff < 0]), 0.75))
slowerTm25.append(np.quantile(abs(diff[diff > 0]), 0.25))
slowerTm.append(np.quantile(abs(diff[diff > 0]), 0.5))
slowerTm75.append(np.quantile(abs(diff[diff > 0]), 0.75))

print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

1455
1455
5059082
5059082
5059082
-----------------------------------------------
843.0
1136.0
2611.0
3112.75
7523.0
12811.25
0.8666599987902943
0.13320282217208576


array([1])

array([1])

11411
11411
11411
868.0
1429.5
2724.0
2342.0
5459.0
10177.75
0.826921391639646
0.17299097362194374


In [16]:
pd.set_option('max_columns', 200)
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]
logSH[(logSH['StockID'] == 869) & (logSH['cum_volume'] <= 5996290)].drop_duplicates(columns, keep='first')[columns +['time']].tail(20)

,StockID,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades,time
5980900,869,5917167,4.320062e+09,2604.4157,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151000000
5981031,869,5929170,4.327042e+09,2605.2325,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151006000
5981274,869,5932178,4.328737e+09,2605.4788,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151012000
5981351,869,5936289,4.330209e+09,2604.7894,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151015000
5981467,869,5936539,4.330323e+09,2605.6346,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151021000
5981614,869,5938005,4.331046e+09,2605.4270,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151027000
5981693,869,5938492,4.331313e+09,2604.9371,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151030000
5981844,869,5938686,4.331425e+09,2604.7789,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151036000
5982014,869,5940116,4.332089e+09,2603.0909,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151042000
5982098,869,5989273,4.360285e+09,2603.4279,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0,151045000


In [12]:
logSH1[(logSH1['StockID'] == 869) & (logSH1['cum_volume'] <= 5996290)].tail(1)[columns]

,StockID,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
5983551,869,5996180,4.365713e+09,2603.3559,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2619.9433,0


In [17]:
logSH[(logSH['StockID'] == 16) & (logSH['cum_volume'] >= 22461640)][columns]

,StockID,cum_volume,close,openPrice
11049699,16,22461640,3485.3772,3480.9266
11050063,16,22461640,3485.3772,3480.9266
11050264,16,22461640,3485.3772,3480.9266
11051415,16,22462530,3485.3772,3480.9266
11051726,16,22462530,3485.4049,3480.9266
11052302,16,22462530,3485.4049,3480.9266
11052654,16,22462530,3485.4049,3480.9266
11052868,16,22462530,3485.4049,3480.9266
11053075,16,22462530,3485.4049,3480.9266
11053449,16,22462530,3485.4049,3480.9266


In [18]:
pd.set_option('max_rows', 200)
logSH1[(logSH1['StockID'] == 16) & (logSH1['cum_volume'] >= 22461640)][columns]

,StockID,cum_volume,close,openPrice
5450434,16,22461640,3485.3772,3480.9266
5450837,16,22461640,3485.3772,3480.9266
5451131,16,22461640,3485.4049,3480.9266
5451456,16,22461640,3485.4049,3480.9266
5452539,16,22462530,3485.4049,3480.9266
5454016,16,22462530,3485.4049,3480.9266
5455493,16,22462530,3485.4049,3480.9266
5456971,16,22462530,3485.4049,3480.9266
5458452,16,22462530,3485.4049,3480.9266
5459938,16,22462530,3485.4049,3480.9266


In [4]:
import pandas as pd

path = np.array(glob.glob('/mnt/data/20201228/zt***/Logs_full/mdTradeLog***'))
assert(len(path) == 3)
print(path[0])
print(path[2])

SH = pd.read_csv(path[0], encoding="utf-8", index_col=False)
SH1 = pd.read_csv(path[2], encoding="utf-8", index_col=False)
SH = SH[SH["exchId"] == 1]
SH1 = SH1[SH1["exchId"] == 1]

import numpy as np
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
# ####
# SH = SH[SH['TransactTime'] <= SH1['TransactTime'].max()]
# ####

columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum", 'secid', 'exchId']
# ####
# SH1 = SH1.drop_duplicates(columns, keep='first')
# ####


n1 = len(SH["SecurityID"].unique())
n2 = len(SH1["SecurityID"].unique())
print(n1)
print(n2)
print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))
print(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
if n1 != n2:
    sl = list(set(SH["SecurityID"].unique()) & set(SH1["SecurityID"].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)
    
# speed check
re1 = re[~np.isnan(re["sequenceNo_x"])]
re1 = re1[~np.isnan(re1["sequenceNo_y"])]
diff = re1["clockAtArrival_y"] - re1["clockAtArrival_x"]
fasterPerc.append(sum(diff < 0)/len(diff))
slowerPerc.append(sum(diff > 0)/len(diff))
fasterTm25.append(np.quantile(abs(diff[diff < 0]), 0.25))
fasterTm.append(np.quantile(abs(diff[diff < 0]), 0.5))
fasterTm75.append(np.quantile(abs(diff[diff < 0]), 0.75))
slowerTm25.append(np.quantile(abs(diff[diff > 0]), 0.25))
slowerTm.append(np.quantile(abs(diff[diff > 0]), 0.5))
slowerTm75.append(np.quantile(abs(diff[diff > 0]), 0.75))

print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

/mnt/data/20201228/zt_7401_20201228_10.235.16.168.tar.gz/Logs_full/mdTradeLog_20201228_0923.csv
/mnt/data/20201228/zt_7401_20201228_10.235.16.80.tar.gz/Logs_full/mdTradeLog_20201228_0923.csv
1453
1453
set()
set()
21777269
21777269
21777269
-----------------------------------------------
1228.0
2125.0
5558.0
2198.0
5415.0
12015.0
0.8566460284804307
0.14334239982065702


In [10]:
SH['TransactTime'].min()

130000000

In [5]:
fr = pd.DataFrame()
fr['cat'] = ['SSE snapshot lv2', 'SSE index', 'SSE trade']
fr['fasterPerc'] = fasterPerc
fr['25fasterTm'] = fasterTm25
fr['fasterTm'] = fasterTm
fr['75fasterTm'] = fasterTm75
fr['25slowerTm'] = slowerTm25
fr['slowerTm'] = slowerTm
fr['75slowerTm'] = slowerTm75
fr['data1'] = 'native api primary'
fr['data2'] = 'native api backup'
fr = fr[['data1', 'data2', 'cat', 'fasterPerc', '25fasterTm', 'fasterTm', '75fasterTm', '25slowerTm', 'slowerTm', '75slowerTm']]
fr['fasterPerc'] = fr['fasterPerc'].apply(lambda x: '%.2f%%'% (x*100))
fr['fasterPerc'] = np.where(fr['fasterPerc'] == '100.00%', '99.99%', fr['fasterPerc'])
for cols in ['25fasterTm', 'fasterTm', '75fasterTm', '25slowerTm', 'slowerTm', '75slowerTm']:
    fr[cols] = fr[cols].astype(int)
fr

,data1,data2,cat,fasterPerc,25fasterTm,fasterTm,75fasterTm,25slowerTm,slowerTm,75slowerTm
0,native api primary,native api backup,SSE snapshot lv2,86.67%,843,1136,2611,3112,7523,12811
1,native api primary,native api backup,SSE index,82.69%,868,1429,2724,2342,5459,10177
2,native api primary,native api backup,SSE trade,85.66%,1228,2125,5558,2198,5415,12015


In [2]:
# # Speed comparison SH
# import pandas as pd
# import numpy as np
# import glob
# import os

# path = np.array(glob.glob('/mnt/data/20201211/***88data.tar.gz/mdLog_SH_***'))
# assert(len(path) == 1)
# path1 = np.array(glob.glob('/mnt/data/20201211/zt_6801_20201211_168.prevde.tar.gz/Logs_full/mdLog_SH_***'))
# assert(len(path1) == 1)
# print(path[0])
# print(path1[0])

# logSH = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
#                                                "exchange", "time", "cum_volume", "cum_amount", "close",
#                                                "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
#                                                "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
#                                                "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
#                                                "ask4q", "ask5q", "openPrice", "numTrades"]]
# logSH1 = pd.read_csv(path1[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
#                                                "exchange", "time", "cum_volume", "cum_amount", "close",
#                                                "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
#                                                "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
#                                                "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
#                                                "ask4q", "ask5q", "openPrice", "numTrades"]]
# print("original data source type: ")
# print(logSH["source"].unique())
# print("new data source type: ")
# print(logSH1["source"].unique())
# print(logSH["time"].unique())
# print(logSH1["time"].unique())

# logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
# logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

# display(logSH['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
# display(logSH1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())

### set here ***
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]

num = 2
### set here ***
    
if num == 2:
    for i in range(0, num):
        in_dex = [16, 300, 852, 905]
        data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == np.sort(logSH['source'].unique())[i])]
        data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == np.sort(logSH1['source'].unique())[i])]
        data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
        data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
        n1 = len(data1_1["StockID"].unique())
        n2 = len(data2_1["StockID"].unique())

        print(n1)
        print(n2)
        if n1 != n2:
            sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
            data1_1 = data1_1[data1_1["StockID"].isin(sl)]
            data2_1 = data2_1[data2_1["StockID"].isin(sl)]
        for cols in ['cum_amount']:
            data1_1[cols] = data1_1[cols].round(2)
            data2_1[cols] = data2_1[cols].round(2)
        data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
        n1 = test["sequenceNo_x"].count()
        n2 = test["sequenceNo_y"].count()
        len1 = len(test)
        print(n1)
        print(n2)
        print(len1)
        print("-----------------------------------------------")
        if n2 < len1:
            display("test is not complete:")
            display(test[np.isnan(test["sequenceNo_y"])])
            print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        if (len1 == n2) & (n1 < len1):
            display("baseline is not complete:")
            display(test[np.isnan(test["sequenceNo_x"])])
            display(n2-n1)
            print((n2-n1)/n1)

else:
    in_dex = [16, 300, 852, 905]
    data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == logSH['source'].max())]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == logSH1['source'].max())]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())

    print(n1)
    print(n2)
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for cols in ['cum_amount']:
        data1_1[cols] = data1_1[cols].round(2)
        data2_1[cols] = data2_1[cols].round(2)
    data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)
 
    

in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]

display(data1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(data2['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())


### set here ***
columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
### set here ***

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

for cols in ["cum_amount"]:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])



import pandas as pd

path = np.array(glob.glob('/mnt/data/20201211/***88data.tar.gz/mdTradeLog_***'))
assert(len(path) == 1)
path1 = np.array(glob.glob('/mnt/data/20201211/zt_6801_20201211_168.prevde.tar.gz/Logs_full/mdTradeLog_***'))
assert(len(path1) == 1)
print(path[0])
print(path1[0])

SH = pd.read_csv(path[0], encoding="utf-8", index_col=False)
SH1 = pd.read_csv(path1[0], encoding="utf-8", index_col=False)
SH = SH[SH["exchId"] == 1]
SH1 = SH1[SH1["exchId"] == 1]

import numpy as np
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
# ####
# SH = SH[SH['TransactTime'] <= SH1['TransactTime'].max()]
# ####

columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum", 'secid', 'exchId']
# ####
# SH1 = SH1.drop_duplicates(columns, keep='first')
# ####


n1 = len(SH["SecurityID"].unique())
n2 = len(SH1["SecurityID"].unique())
print(n1)
print(n2)
print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))
print(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
if n1 != n2:
    sl = list(set(SH["SecurityID"].unique()) & set(SH1["SecurityID"].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)


1756
1458
4894537
4894537
4894537
-----------------------------------------------
1756
1456
5192355
5192355
5192355
-----------------------------------------------


array([1, 5])

array([1, 5])

11407
11407
11407
/mnt/data/20201211/logs_20201211_zt_88_03_day_88data.tar.gz/mdTradeLog_20201211_0832.csv
/mnt/data/20201211/zt_6801_20201211_168.prevde.tar.gz/Logs_full/mdTradeLog_20201211_0857.csv
1756
1456
{688128, 688129, 605188, 688133, 688135, 601609, 688138, 688139, 688028, 688122, 603662, 605198, 605199, 688029, 603155, 688030, 688155, 688156, 688157, 688158, 688159, 688160, 603682, 605218, 688165, 688166, 603687, 688168, 688169, 605222, 688177, 688178, 688179, 688180, 688181, 688037, 688185, 688186, 601658, 688188, 688189, 603195, 688039, 688196, 688198, 688199, 688200, 603719, 688202, 605255, 603212, 605258, 688208, 603221, 688215, 688218, 688219, 688221, 688222, 601696, 601187, 688228, 688229, 601702, 603236, 605288, 688233, 603755, 603256, 688051, 688256, 688258, 605318, 688127, 688266, 603786, 688268, 603279, 688788, 688277, 688278, 601236, 605333, 605336, 603290, 605338, 688286, 688288, 688289, 688058, 603815, 688298, 688299, 688300, 688301, 605358, 688060, 601778, 68830